In [1]:
import time
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
#imports
#RUN ONLY 1 TIME: collects 'initial odds': therefore all differences will be 0 at time 0, as time goes on since running (and games start), the odds change
import requests
import pandas as pd
from pandas.io.json import json_normalize
from functools import reduce
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime

#function specific to json ex: https://sportsbook.fanduel.com/cache/psmg/US/53474.3.json
def parse_data(jsooooon):
    results = pd.DataFrame()
    for ayo in jsooooon['events']:
        money = json_normalize(ayo).drop('markets',axis=1)
        for bandos in ayo['markets']:
            gambling = json_normalize(bandos).drop('selections',axis=1)
            gambling.columns = [str(col) + '.markets' for col in gambling.columns]
            for theta in bandos['selections']:
                yeye = json_normalize(theta)
                yeye.columns = [str(col) + '.selections' for col in yeye.columns]
                yoyo = reduce(lambda left,right: pd.merge(left,right, left_index=True, right_index=True), [money, gambling, yeye])
                results = results.append(yoyo, sort=True).reset_index(drop=True)
    return results

#getting jsons, nfl doesnt work
nfl = requests.get('https://sportsbook.fanduel.com/cache/psmg/UK/57494.3.json').json()
nba = requests.get('https://sportsbook.fanduel.com/cache/psmg/UK/55978.3.json').json()
bball = requests.get('https://sportsbook.fanduel.com/cache/psmg/US/53474.3.json').json()


#setting up df
cbb = parse_data(bball)
cb = pd.DataFrame()
cbb_ = cbb[cbb['name.markets']=='Spread Betting']
cb[['lines','team','away_t','home_t','start_time']] = cbb_[['currenthandicap.selections','name.selections','participantname_away','participantname_home','tsstart']]

away = cb[cb['team']==cb['away_t']]
away['lines'] = away['lines'].astype(int)*-1
home = cb[cb['team']==cb['home_t']]

#RUN THIS CELL, THEN PREP WORK DONE (DONT RUN AGAIN!) --> ALL OF YOUR DIFFERENCES WILL BE 0 at first!!
home_initial = pd.DataFrame()
away_initial = pd.DataFrame()
away_initial[['initial_lines','team']] = away[['lines','team']]
home_initial[['initial_lines','team']] = home[['lines','team']]
away_initial.head()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,initial_lines,team
6,7,Robert Morris
8,-4,Northern Kentucky
21,-1,UConn
28,15,Maryland
33,7,Detroit Mercy


In [3]:
def game_lines(time_mins):
    nfl = requests.get('https://sportsbook.fanduel.com/cache/psmg/UK/57494.3.json').json()
    nba = requests.get('https://sportsbook.fanduel.com/cache/psmg/UK/55978.3.json').json()
    bball = requests.get('https://sportsbook.fanduel.com/cache/psmg/US/53474.3.json').json()


    #setting up df
    cbb = parse_data(bball)
    cb = pd.DataFrame()
    cbb_ = cbb[cbb['name.markets']=='Spread Betting']
    cb[['lines','team','away_t','home_t','start_time']] = cbb_[['currenthandicap.selections','name.selections','participantname_away','participantname_home','tsstart']]

    away = cb[cb['team']==cb['away_t']]
    away['lines'] = away['lines'].astype(int)*-1
    home = cb[cb['team']==cb['home_t']]

    away = pd.merge(away,away_initial,on='team')
    home = pd.merge(home,home_initial,on='team')
    home['difference'] = home['initial_lines'].astype(int) - home['lines'].astype(int)
    away['difference'] = away['initial_lines'].astype(int) - away['lines'].astype(int)
    pieces = (away,home)
    all_games_today = pd.concat(pieces, ignore_index = True)
    all_games_today = all_games_today[all_games_today['start_time']>='2022-01-26T19:50:00']

    now = datetime.now()
    all_games_today['current_time'] = now
    all_games_today['datethen'] = pd.to_datetime(all_games_today['current_time'],infer_datetime_format = True)
    all_games_today['start_time'] = pd.to_datetime(all_games_today['start_time'],infer_datetime_format = True)
    all_games_today['time_from_start'] = all_games_today['datethen']-all_games_today['start_time']

    all_games_today = all_games_today.sort_values('difference')

    all_games_today['rank'] = all_games_today['difference']
    all_games_today = all_games_today.set_index('rank')
    return all_games_today       


In [46]:
superbowl = 'http://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard'
import requests as re
df = re.get(superbowl).json()
df = df['events']
for s in df:
    df = s['competitions']
aa = {'a':[]}
for d in df:
    aa['a'].append(d['competitors'])
df = aa['a']

aa = {'a':[]}
for d in df1:
    aa['a'].append(d['records'])
df = aa['a']

df

[[{'name': 'All Splits',
   'abbreviation': 'Any',
   'type': 'total',
   'summary': '10-7'},
  {'name': 'Home', 'type': 'home', 'summary': '5-4'},
  {'name': 'Road', 'type': 'road', 'summary': '5-3'}],
 [{'name': 'All Splits',
   'abbreviation': 'Any',
   'type': 'total',
   'summary': '12-5'},
  {'name': 'Home', 'type': 'home', 'summary': '5-3'},
  {'name': 'Road', 'type': 'road', 'summary': '7-2'}]]

In [109]:
superbowl = 'http://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard'
import requests as re
df = re.get(superbowl).json()
df = df['events']
for s in df:
    df = s['competitions']
aa = {'a':[]}
for d in df:
    aa['a'].append(d['competitors'])
df = aa['a']

aa = {'a':[]}
for d in df1:
    aa['a'].append(d['leaders'])
df = aa['a']

away = df[1]
home = df[0]


away_stats ={'name':[],'position':[],'stats':[]}

away_qb = away[0]
away_rush = away[1]
away_rush = away_rush['leaders']
for s in away_rush:
    away_stats['stats'].append(s['displayValue'])
    away_rush = s['athlete']
    away_stats['name'].append(away_rush['fullName'])
    away_rush = away_rush['position']
    away_stats['position'].append(away_rush['abbreviation'])

away_recep = away[2]
away_recep = away_recep['leaders']

for s in away_recep:
    away_stats['stats'].append(s['displayValue'])
    away_recep = s['athlete']
    away_stats['name'].append(away_recep['fullName'])
    away_recep = away_recep['position']
    away_stats['position'].append(away_recep['abbreviation'])

away_stats = pd.DataFrame(away_stats)
away_stats

,name,position,stats
0,Sony Michel,RB,"208 CAR, 845 YDS, 4 TD"
1,Cooper Kupp,WR,"145 REC, 1947 YDS, 16 TD"


In [8]:
espn_scoreboard = 'http://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard'
def moreStats(df):
    for s in df:
        df = s['competitors']
    
    a = {'a' :[]}
    for name in df:
        a['a'].append(name['team'])



    df_0 = df[0]
    df_0_name = a['a'][0]['displayName']
    df_1 = df[1]
    df_1_name = a['a'][1]['displayName']


    df_0 = df_0['statistics']
    df_1 = df_1['statistics']


    team_1 = {'name':[],'value':[]}
    team_2 = {'name':[],'value':[]}



    for s in df_0:
        team_1['name'].append(s['name'])
        team_1['value'].append(s['displayValue'])

    team_1 = pd.DataFrame(team_1)

    for s in df_1:
        team_2['name'].append(s['name'])
        team_2['value'].append(s['displayValue'])
    



    team_2 = pd.DataFrame(team_2)
    teams = pd.merge(team_1,team_2, on = 'name')

    df = pd.DataFrame()
    df[df_0_name] = teams['value_x']
    df[df_1_name] = teams['value_y']
    df['stat'] = teams['name']
    return df

def live_stats(espn_scoreboard):    
    import requests as re
    df = re.get(espn_scoreboard).json()
    df = df['events']
    dff= {'a':[]}
    for s in df:
        dff['a'].append(s['competitions'])
    df = dff['a']
    dfff = {'first':[],'second':[],'third':[],'fourth':[],'fifth':[],'sixth':[],'seventh':[],'eighth':[],'ninth':[]}
    for nine in df:
        dfff['first'].append(df[0])
        dfff['second'].append(df[1])
        dfff['third'].append(df[2])
        dfff['fourth'].append(df[3])
        dfff['fifth'].append(df[4])
        dfff['sixth'].append(df[5])
        #dfff['seventh'].append(df[6])
        #dfff['eighth'].append(df[7])
        #dfff['ninth'].append(df[8])


    first = moreStats(dfff['first'][0])
    first1 = moreStats(dfff['second'][0])
    first2 = moreStats(dfff['third'][0])
    first3 = moreStats(dfff['fourth'][0])
    first4 = moreStats(dfff['fifth'][0])
    first5 = moreStats(dfff['sixth'][0])
    #first6 = moreStats(dfff['seventh'][0])
    #first7 = moreStats(dfff['eighth'][0])
    #first8 = moreStats(dfff['ninth'][0])


    smu_v_houston = first
    rutgers_v_ohioState = first1
    setonHall_v_Xavier = first2
    kansasState_v_Baylor = first3
    oklahoma_v_texasTech= first4
    mississippiState_v_tennessee = first5
    
    all_games = pd.merge( smu_v_houston, rutgers_v_ohioState,on = 'stat')
    all_games = pd.merge(all_games ,setonHall_v_Xavier ,on = 'stat')
    all_games = pd.merge(all_games , kansasState_v_Baylor, on = 'stat')
    all_games = pd.merge( all_games, oklahoma_v_texasTech, on = 'stat')
    all_games = pd.merge( all_games,mississippiState_v_tennessee ,on = 'stat')


    all_games = all_games.set_index('stat')

    
    return all_games

In [5]:
def name(espn):
    df = re.get(espn).json()
    df = df['sports']
    newusers = dict()
    for ud in df:
        newusers[ud.pop('id')] = ud
    dd = newusers['40']
    newnew = dict()
    new = dd['leagues']
    for xx in new:
        new = xx['teams']
    aa = {0 :[],1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[],11:[],12:[],13:[],14:[],15:[],16:[]}
    nn = {'name':[]}
    for i in range(len(new)):
        newnew = new[i]
        nw = newnew['team']
        nn['name'].append(nw['displayName'])
    nn = nn['name']
    return nn

def pre_game_stats(espn):
    df = re.get(espn).json()
    df = df['sports']
    newusers = dict()
    for ud in df:
        newusers[ud.pop('id')] = ud
    dd = newusers['40']
    newnew = dict()
    new = dd['leagues']
    for xx in new:
        new = xx['teams']
    aa = {0 :[],1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[],11:[],12:[],13:[],14:[],15:[],16:[]}
    for i in range(len(new)):
        newnew = new[i]
        nw = newnew['team']
        nww = nw['record']
        nwww = nww['items']
        for summ in nwww:
            nwww = summ['stats']
        nwww = pd.DataFrame(nwww)
        aa[i].append(nwww['value'])
    a1 = pd.DataFrame(aa[0])
    a2= pd.DataFrame(aa[1])
    a3= pd.DataFrame(aa[2])
    a4= pd.DataFrame(aa[3])
    a5= pd.DataFrame(aa[4])
    a6= pd.DataFrame(aa[5])
    a7= pd.DataFrame(aa[6])
    a8= pd.DataFrame(aa[7])
    a9= pd.DataFrame(aa[8])
    a10= pd.DataFrame(aa[9])
    a11= pd.DataFrame(aa[10])
    a12= pd.DataFrame(aa[11])
    a13= pd.DataFrame(aa[12])
    a14= pd.DataFrame(aa[13])
    a15= pd.DataFrame(aa[14])
    a16= pd.DataFrame(aa[15])
    a17= pd.DataFrame(aa[16])
    p = (a1,a2, a3 ,a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15, a16, a17)
    a = pd.concat(p, ignore_index = True)
    aaa = pd.DataFrame()
    aaa['name'] = nn
    aaa['playoffSeed'] = a[0]
    aaa['wins'] = a[1]
    aaa['losses'] = a[2]
    aaa['winPercent'] = a[3]
    aaa['gamesBehind'] = a[4]
    aaa['ties'] = a[5]
    aaa['OTWins'] = a[6]
    aaa['OTLosses'] = a[7]
    aaa['gamesPlayed'] = a[8]
    aaa['pointsFor'] = a[9]
    aaa['pointsAgainst'] = a[10]
    aaa['avgPointsFor'] = a[11]
    aaa['avgPointsAgainst'] = a[12]
    aaa['points'] = a[13]
    aaa['differential'] = a[14]
    aaa['streak'] = a[15]
    aaa['divisionWinPercent'] = a[16]
    aaa['leagueWinPercent'] = a[17]
    return aaa

In [6]:
from numpy import random
import time
time_mins = 1000

In [ ]:
for i in range(time_mins):
  glines = game_lines(time_mins)
  glines.to_csv(f'NCAAB_GAMES_Feb13.csv',index = True)
  time.sleep(random.uniform(60,300
                            ))



//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [39]:
for i in range(time_mins):
  all_games = live_stats(espn_scoreboard)
  all_games.to_csv(f'{PATH}all_games.csv',index = False)
  time.sleep(random.uniform(10,30))

In [31]:
espn = 'http://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/teams'

for i in range(time_mins):
  nn = name(espn)
  pregame = pre_game_stats(espn)
  pregame.to_csv(f'{PATH}pregame_stats.csv',index = False)
  time.sleep(random.uniform(10,30))

In [13]:
#enter this to run it
current_time = list()
cb = pd.DataFrame()

for i in range(time_mins):
    i = [i]
    nfl = requests.get('https://sportsbook.fanduel.com/cache/psmg/UK/57494.3.json').json()
    nba = requests.get('https://sportsbook.fanduel.com/cache/psmg/UK/55978.3.json').json()
    bball = requests.get('https://sportsbook.fanduel.com/cache/psmg/US/53474.3.json').json()


    #setting up df
    cbb = parse_data(bball)
    cbb_ = cbb[cbb['name.markets']=='Spread Betting']
    cb[['lines'+str(i),'team','away_t','home_t','start_time']] = cbb_[['currenthandicap.selections','name.selections','participantname_away','participantname_home','tsstart']]

    away = cb[cb['team']==cb['away_t']]
    away['lines'+str(i)] = away['lines'+str(i)].astype(int)*-1
    home = cb[cb['team']==cb['home_t']]

    away = pd.merge(away,away_initial,on='team')
    home = pd.merge(home,home_initial,on='team')
    home['difference'] = home['initial_lines'].astype(int) - home['lines'+str(i)].astype(int)
    away['difference'] = away['initial_lines'].astype(int) - away['lines'+str(i)].astype(int)
    pieces = (away,home)
    all_games_today = pd.concat(pieces, ignore_index = True)
    all_games_today = all_games_today[all_games_today['start_time']>='2022-01-25T15:20:00']
    
    now = datetime.now()
    all_games_today['current_time'] = now
    all_games_today['datethen'] = pd.to_datetime(all_games_today['current_time'],infer_datetime_format = True)
    all_games_today['start_time'] = pd.to_datetime(all_games_today['start_time'],infer_datetime_format = True)
    all_games_today['time_from_start'] = all_games_today['datethen']-all_games_today['start_time']
    
    all_games_today_away = all_games_today[all_games_today['team']==all_games_today['away_t']]
    all_games_today_home = all_games_today[all_games_today['team']==all_games_today['home_t']]

    all_games_today_away = all_games_today_away.sort_values('difference')
    all_games_today_home = all_games_today_home.sort_values('difference')

    
    if len(all_games_today_home)>3:
        all_games_today_home.to_csv('NCAAB_HOMEGAMES_FEB9.csv',index = False)
    if len(all_games_today_away)>3:
        all_games_today_away.to_csv('NCAAB_AWAYGAMES_FEB9.csv',index = False)

    time.sleep(random.uniform(60,120))


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
//anac

KeyError: 'events'

In [ ]:
import pandas as pd

In [29]:
import requests as re